# Scapy

### Install

In [1]:
!pip install scapy

     |████████████████████████████████| 1.1 MB 3.2 MB/s eta 0:00:01
  Created wheel for scapy: filename=scapy-2.4.5-py2.py3-none-any.whl size=1261545 sha256=f80a7c0419449ad51c3cf850f9ad6f83ac1b4657ef6660e9c4781177e1936996
  Stored in directory: /home/tn/.cache/pip/wheels/c8/9b/2f/012f0dbaf869afac8be52d4423f34eaa0b6c5c2d1292e40ebd
Successfully built scapy


### Basics

In [14]:
from scapy.all import *

# create a basic IP-Packet
IP().show()

###[ IP ]### 
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = hopopt
  chksum    = None
  src       = 127.0.0.1
  dst       = 127.0.0.1
  \options   \



#### simple network scanner

In [15]:
!ip address

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: wlp2s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state UP group default qlen 1000
    link/ether c4:8e:8f:f3:9b:a7 brd ff:ff:ff:ff:ff:ff
    inet 192.168.178.22/24 brd 192.168.178.255 scope global dynamic noprefixroute wlp2s0
       valid_lft 861500sec preferred_lft 861500sec
    inet6 2001:4dd6:daa9:0:b453:1252:c7cf:e802/64 scope global temporary dynamic 
       valid_lft 6827sec preferred_lft 3227sec
    inet6 2001:4dd6:daa9:0:15da:fa06:9ada:62ef/64 scope global dynamic mngtmpaddr noprefixroute 
       valid_lft 6827sec preferred_lft 3227sec
    inet6 fe80::470:6b4c:4354:bc76/64 scope link noprefixroute 
       valid_lft forever preferred_lft forever


In [23]:
from scapy.all import ARP, Ether, srp
# set target IP range
target = "192.168.178.1/24"
# create ARP (Address Resolution protocol) packet
arp = ARP(pdst=target)
ether = Ether(dst="ff:ff:ff:ff:ff:ff")
packet = ether/arp

# raw socket operations require elevated privileges
result = srp(packet, timeout=30)

PermissionError: [Errno 1] Operation not permitted